In [2]:

import PyPDF2 #importing the library
pdf_file_obj = open('invoice.pdf', 'rb')# opening the pdf file in read binary format
pdf_reader = PyPDF2.PdfReader(pdf_file_obj) # creating pdf reader object
num_pages = len(pdf_reader.pages) # reading the total pages


In [3]:
detected_text=''
for pages in range(num_pages):
    page_obj = pdf_reader.pages[pages] # reading the particular page
    detected_text+=page_obj.extract_text()+'\n\n' # extracting the content of page and saving in the created string
    
pdf_file_obj.close() # clossing the file 
print(detected_text)

Tax Invoice
Order Id: 
OD226236519855920000 
Order Date: 
11-10-2022, 11:09 PM
Invoice No: 
FAAWJR2300643139
Invoice Date: 
13-10-2022, 02:16 PM
GSTIN: 29AADCP7081J1ZY
PAN: AADCP7081J
Sold By
 
PUMA Sports India Pvt Ltd,
Sy No. 491,
492/3,492/4,494,493/1,493/2,493/3,493/4,493/5,513/1,513/2,516/1,516/2,517/1,516/5,492,,
Madivala village,Kasaba Hobli, Anekal Taluk,
BANGALORE - 562106
 
GST: 29AADCP7081J1ZY 
Billing Address
 
Sachin 
Saini,
Gali no 3, Near Om Sai Mobile,
Dayanand Nagar,
Bahadurgarh - 
124507, 
IN-HR 
Shipping ADDRESS
Sachin Saini, 
Gali no 3, Near Om Sai Mobile,
Dayanand Nagar,
Bahadurgarh - 124507, IN-HR
Seller Registered Address:
 
PUMA Sports India Pvt Ltd,
Puma sports India Pvt Ltd/ Word Cat India Pvt Ltd. 496, Mahadevapura Main Road, 
, 
Next to Hewlett Packard Service Gate, Mahadevapura,
Bangalore, 
BANGALORE - 560048. 
Declaration 
The goods sold are intended for end user consumption and not for resale. 
E. & O.E.
Product
Description
Qty
Gross 
Amount
Discount
Taxa

In [5]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("openai_base"), 
  api_key=os.getenv("api_key"),  
  api_version=os.getenv("api_version")
)

def fun(text,Persona=None):
  if Persona is None:
    message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},
                {"role":"user","content":text}
                ]
  else:
    message_text = [{"role":"system","content":Persona},
                {"role":"user","content":text}
                ]  
  completion = client.chat.completions.create(
  model="GPT4-Turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.8,
  max_tokens=1000,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)
  print("Completion Tokens:",completion.usage.completion_tokens)
  print("Input Tokens :",completion.usage.prompt_tokens)
  return completion.choices[0].message.content.strip()
  

In [6]:
persona=" You are a invoice processing solution."

In [42]:
query=''' Extract data from below invoice of a shoe and return only json containing these following details:
invoice_date, invoice_number, seller_name, seller_address, total_amount, and each line item present in the invoice.
Invoice:\n {invoice}
Note:
give json only in output.
Don't start with 'json'
'''

In [43]:
json1=fun(query.format(invoice=detected_text),persona)

Completion Tokens: 318
Input Tokens : 610


In [45]:
print(json1)

{
  "invoice_date": "13-10-2022, 02:16 PM",
  "invoice_number": "FAAWJR2300643139",
  "seller_name": "PUMA Sports India Pvt Ltd",
  "seller_address": "Sy No. 491, 492/3,492/4,494,493/1,493/2,493/3,493/4,493/5,513/1,513/2,516/1,516/2,517/1,516/5,492,, Madivala village,Kasaba Hobli, Anekal Taluk, BANGALORE - 562106",
  "total_amount": 1899.00,
  "line_items": [
    {
      "product_description": "PUMA Grandeur Walking Shoes For Men Black-White 8 377274",
      "quantity": 1,
      "gross_amount": 1899.00,
      "discount": 0.00,
      "taxable_value": 1609.32,
      "IGST": 289.68,
      "total": 1899.00
    },
    {
      "product_description": "Shipping and Handling Charges",
      "quantity": 1,
      "gross_amount": 0.00,
      "discount": 0,
      "taxable_value": 0.00,
      "IGST": 0.00,
      "total": 0.00
    }
  ]
}


In [46]:
import json

In [47]:
json.loads(json1)

{'invoice_date': '13-10-2022, 02:16 PM',
 'invoice_number': 'FAAWJR2300643139',
 'seller_name': 'PUMA Sports India Pvt Ltd',
 'seller_address': 'Sy No. 491, 492/3,492/4,494,493/1,493/2,493/3,493/4,493/5,513/1,513/2,516/1,516/2,517/1,516/5,492,, Madivala village,Kasaba Hobli, Anekal Taluk, BANGALORE - 562106',
 'total_amount': 1899.0,
 'line_items': [{'product_description': 'PUMA Grandeur Walking Shoes For Men Black-White 8 377274',
   'quantity': 1,
   'gross_amount': 1899.0,
   'discount': 0.0,
   'taxable_value': 1609.32,
   'IGST': 289.68,
   'total': 1899.0},
  {'product_description': 'Shipping and Handling Charges',
   'quantity': 1,
   'gross_amount': 0.0,
   'discount': 0,
   'taxable_value': 0.0,
   'IGST': 0.0,
   'total': 0.0}]}

## Document Classification

In [49]:
import shutil
import os

In [ ]:
def list_files_only(directory_path):
    if os.path.isdir(directory_path):
        